In [1]:
import os, gc
from tqdm import tqdm

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate
from langchain.vectorstores import Chroma, FAISS
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline
#from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA

import torch

import warnings
warnings.filterwarnings('ignore')

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
NVIDIA GeForce RTX 3080 Ti


In [2]:
loader = PyPDFLoader("pdf_samples/1716891429607.pdf")
pages = loader.load_and_split()

print(pages[11].page_content)

II БҮЛЭГ.ӨГӨГДЛИЙН ХҮСНЭГТ
ТШЗ-ны 
холбогдох 
заалтА.ЕРӨНХИЙ ЗҮЙЛ
ТШЗ 1.1Захиалагчийн нэр: Монгол шуудан ХК
ТШЗ 1.1Зарлалын дугаар: МШ/20240202070
Тендер шалгаруулалтын нэр: Принтерийн хор
Тендер шалгаруулалтын багцын тоо болон нэр: Байхгүй
ТШЗ 2.1Санхүүжилтийн эх үүсвэр: Өөрийн хөрөнгө
Нийт төсөвт өртөг: 43,000,000 /дөчин гурван сая / төгрөг
Б.АГУУЛГА
ТШЗ 9.2Тендерийн баримт бичгийн талаар тодруулга авах хаяг:
Тендерийн баримт бичгийн талаар тодруулга авах хүсэлтийг хүлээн авах  
хугацааны хязгаар тендер хүлээн авах эцсийн хугацаанаас ажлын 2-оос  
доошгүй  хоногийн  өмнө  байна.  Тодруулга,  нэмэлт  мэдээлэл  авах  
хүсэлтийн хариуг хүсэлт гаргагчийн нэрийг дурдалгүйгээр албан бичгээр  
гаргаж,  түүний  хуулбарыг  Төрийн  худалдан  авах  ажиллагааны  цахим  
систем  (www.tender.gov.mn)-д  нийтлэх  бөгөөд  тус  тодруулга,  нэмэлт  
мэдээллийг бүх тендерт оролцогчдод нэгэн зэрэг хүргүүлсэнд тооцно.  
Иймд тендерт оролцохыг сонирхогч этгээдүүд дээрх цахим систем дэх  
тендерийн  урилгын

In [3]:
docs = loader.load()

r_splitter = RecursiveCharacterTextSplitter(chunk_size = 10000, chunk_overlap = 100)
docs_split = r_splitter.split_documents(docs)

In [4]:
embedding = HuggingFaceEmbeddings(
    model_name = "tugstugi/bert-large-mongolian-uncased",
    model_kwargs = {"device": "cuda"},
    encode_kwargs = {"normalize_embeddings": True}
)

No sentence-transformers model found with name tugstugi/bert-large-mongolian-uncased. Creating a new one with mean pooling.
Some weights of BertModel were not initialized from the model checkpoint at tugstugi/bert-large-mongolian-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
vector_store = Chroma.from_documents(documents = docs_split, 
                                     embedding = embedding,
                                     persist_directory = 'db/')

../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [108,0,0], thread: [32,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [108,0,0], thread: [33,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [108,0,0], thread: [34,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [108,0,0], thread: [35,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [108,0,0], thread: [36,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [108,0,0], thread: [37,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1289: indexSelectLargeIndex: block: [108,

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
